<a href="https://colab.research.google.com/github/DataQuest123/Descriptive-Statistics/blob/main/Standalone_Dash_App_(shareable_web_UI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# dash_transform_app.py
import numpy as np
from dash import Dash, dcc, html, Input, Output, State
import plotly.graph_objects as go

# Presets (reuse small set)
PRESETS_2D = {
    'Identity': np.array([[1.,0.],[0.,1.]]),
    'Rotation 45°': np.array([[np.cos(np.pi/4), -np.sin(np.pi/4)],
                              [np.sin(np.pi/4),  np.cos(np.pi/4)]]),
    'Scaling (2x, 0.5y)': np.array([[2.,0.],[0.,0.5]]),
    'Shear x by 1': np.array([[1.,1.],[0.,1.]])
}

def make_grid_2d(range_min=-2, range_max=2, points=15):
    x = np.linspace(range_min, range_max, points)
    y = np.linspace(range_min, range_max, points)
    X, Y = np.meshgrid(x, y)
    pts = np.vstack([X.flatten(), Y.flatten()])
    return X, Y, pts

def apply_transform(A, pts):
    return A @ pts

def plotly_fig_2d(A, grid_range=3, grid_pts=15, show_grid=True, show_basis=True):
    X, Y, pts = make_grid_2d(range_min=-grid_range, range_max=grid_range, points=grid_pts)
    transformed = apply_transform(A, pts)
    X_t = transformed[0,:].reshape(X.shape)
    Y_t = transformed[1,:].reshape(Y.shape)
    fig = go.Figure()
    if show_grid:
        for i in range(X.shape[0]):
            fig.add_trace(go.Scatter(x=X[i,:], y=Y[i,:], mode='lines', line=dict(width=1), showlegend=False, opacity=0.5))
        for j in range(X.shape[1]):
            fig.add_trace(go.Scatter(x=X[:,j], y=Y[:,j], mode='lines', line=dict(width=1), showlegend=False, opacity=0.5))
        for i in range(X_t.shape[0]):
            fig.add_trace(go.Scatter(x=X_t[i,:], y=Y_t[i,:], mode='lines', line=dict(width=1), showlegend=False))
        for j in range(X_t.shape[1]):
            fig.add_trace(go.Scatter(x=X_t[:,j], y=Y_t[:,j], mode='lines', line=dict(width=1), showlegend=False))
    if show_basis:
        E = np.eye(2)
        T = A @ E
        for i in range(2):
            fig.add_trace(go.Scatter(x=[0, E[0,i]], y=[0, E[1,i]], mode='lines+markers', name=f'e{i+1}'))
            fig.add_trace(go.Scatter(x=[0, T[0,i]], y=[0, T[1,i]], mode='lines+markers', name=f'Ae{i+1}'))
    fig.update_layout(xaxis=dict(scaleanchor='y', range=[-grid_range*2, grid_range*2]),
                      yaxis=dict(range=[-grid_range*2, grid_range*2]),
                      width=700, height=700)
    return fig

app = Dash(__name__)
app.layout = html.Div([
    html.H2("Linear Transformation Visualizer (2D) - Dash"),
    html.Div([
        html.Label("Preset"),
        dcc.Dropdown(options=list(PRESETS_2D.keys()), id='preset'),
    ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    html.Div([
        html.Label("Matrix entries (2x2)"),
        html.Div([
            dcc.Input(id='a00', type='number', value=1, step=0.01, style={'width':'80px'}),
            dcc.Input(id='a01', type='number', value=0, step=0.01, style={'width':'80px', 'marginLeft':'8px'}),
            html.Br(),
            dcc.Input(id='a10', type='number', value=0, step=0.01, style={'width':'80px', 'marginTop':'8px'}),
            dcc.Input(id='a11', type='number', value=1, step=0.01, style={'width':'80px', 'marginLeft':'8px'})
        ])
    ], style={'display':'inline-block', 'marginLeft':'20px'}),
    html.Div([
        html.Label("Grid range"),
        dcc.Slider(1,6,1, value=3, id='grid-range')
    ], style={'width':'40%', 'marginTop':'20px'}),
    dcc.Checklist(options=[{'label':'Show grid','value':'g'},{'label':'Show basis','value':'b'}], value=['g','b'], id='flags'),
    dcc.Graph(id='transform-plot'),
    html.Div(id='det-info', style={'marginTop':'10px', 'whiteSpace':'pre-wrap'})
])

@app.callback(
    Output('a00', 'value'), Output('a01', 'value'), Output('a10', 'value'), Output('a11', 'value'),
    Input('preset', 'value')
)
def load_preset(preset):
    if not preset:
        return 1,0,0,1
    M = PRESETS_2D[preset]
    return float(M[0,0]), float(M[0,1]), float(M[1,0]), float(M[1,1])

@app.callback(
    Output('transform-plot','figure'),
    Output('det-info','children'),
    Input('a00','value'), Input('a01','value'), Input('a10','value'), Input('a11','value'),
    Input('grid-range','value'),
    Input('flags','value')
)
def update_plot(a00,a01,a10,a11, grid_range, flags):
    A = np.array([[a00 or 0.0, a01 or 0.0],[a10 or 0.0, a11 or 0.0]])
    show_grid = 'g' in (flags or [])
    show_basis = 'b' in (flags or [])
    fig = plotly_fig_2d(A, grid_range=grid_range, grid_pts=15, show_grid=show_grid, show_basis=show_basis)
    det = np.linalg.det(A)
    try:
        inv = np.linalg.inv(A)
        inv_str = np.array2string(inv, precision=3, suppress_small=True)
    except:
        inv_str = "None (singular)"
    det_info = f"Determinant: {det:.5f}\nInverse:\n{inv_str}"
    return fig, det_info

if __name__ == '__main__':
    print("Running Dash app on http://127.0.0.1:8050")
    app.run(debug=True)

Running Dash app on http://127.0.0.1:8050


<IPython.core.display.Javascript object>